# Azure OpenAI Service Load Balancing with Azure API Management

This notebook demonstrates how to use Azure API Management to load balance requests to multiple deployed Azure OpenAI services.

## Prerequisites

The notebook uses [PowerShell](https://learn.microsoft.com/powershell/scripting/install/installing-powershell) and [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) to deploy all necessary Azure resources. Both tools are available on Windows, macOS and Linux environments.

Running this notebook will deploy the following resources in your Azure subscription:
- Azure Resource Group
- Azure Managed Identity
- Azure Key Vault
- Azure OpenAI Service (West Europe + East US)
- Azure API Management

## 1. Login to Azure & set subscription

The following will prompt you to login to Azure. Once logged in, the current default subscription in your available subscriptions will be set for deployment.

> **Note:** If you have multiple subscriptions, you can change the default subscription by running `az account set --subscription <subscription_id>`.

In [ ]:
# Check if you are already logged in
$loggedIn = az account show --query "name" -o tsv

if ($loggedIn -ne $null) {
    Write-Host "Already logged in as $loggedIn"
} else {
    Write-Host "Logging in..."
    az login
}

# Retrieve ID for current subscription
$subscriptionId = (
    (
        az account list -o json `
            --query "[?isDefault]"
    ) | ConvertFrom-Json
).id

# Set subscription ID
az account set --subscription $subscriptionId
Write-Host "Subscription set to $subscriptionId"

## 2. Deploy Azure resources with Bicep

The following will deploy all the necessary Azure resources, previously listed, using [Azure Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/).

The deployment occurs at the subscription level, creating a new resource group. The location of the deployment is set to **West Europe** and this can be changed, as well as other parameters, in the [`./infra/main.bicepparam`](./infra/main.bicepparam) file.

> **Note:** To run this deployment successfully, you must provide a value for the `apiManagementPublisherEmail` and `apiManagementPublisherName` parameters in the [`./infra/main.bicepparam`](./infra/main.bicepparam) file. These values are used to configure the Azure API Management instance.

This may take up to 10 minutes due to the Azure API Management resource needing to be activated successfully.

### Understanding the deployment

#### Managed Identity

A [user-assigned Managed Identity](https://learn.microsoft.com/en-us/entra/identity/managed-identities-azure-resources/overview) is created for the Azure API Management instance. This is used to authenticate with the Azure Key Vault instance to retrieve the API Keys for the Azure OpenAI Service instances.

#### Key Vault

An [Azure Key Vault](https://learn.microsoft.com/en-us/azure/key-vault/general/overview) instance is created to store the API Keys for the deployed Azure OpenAI Service instances. The API Keys are stored as [secrets](https://learn.microsoft.com/en-us/azure/key-vault/secrets/about-secrets) in the Key Vault instance.

#### OpenAI Services

Two [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/cognitive-services/openai-service/overview) instances are deployed, one in the West Europe region and one in the East US region. These are deployed with the `gpt-35-turbo` models to be used for inference.

#### API Management - Backends

[API Management backends](https://learn.microsoft.com/en-us/azure/api-management/backends?tabs=bicep) are created for each deployed Azure OpenAI Service instance. Each backend points to the endpoint of the deployed Azure OpenAI Service instances so that we can use them in conjunction with the deployed API Management API.

#### API Management - API

Azure OpenAI Service has a standard REST API specification that we can use to import the API into API Management. You can [find the latest OpenAPI specifications in the Azure REST API specifications GitHub repository](https://github.com/Azure/azure-rest-api-specs/tree/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/preview).

The API is configured on the `/openai` path for the deployed Azure API Management instance.

#### API Management - Subscription

In order to access the API, we need to [create a subscription for it](https://learn.microsoft.com/en-us/azure/api-management/api-management-subscriptions). This will generate a subscription key that we can use to make requests to the API.

#### API Management - Named Values

Based on the deployed Azure OpenAI Service instances, [API Management Named Values](https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-properties?tabs=azure-portal) are created for each API Key that was stored in Azure Key Vault as part of the deployment.

We create these so that we can use them in the API Management policies when making requests to the Azure OpenAI Service.

#### API Management - Policy

With everything configured in Azure API Management to take advantage of the multiple Azure OpenAI Service instances, we [deploy a policy to the API Management API](https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-policies) to load balance requests across the backends.

The [API Management OpenAI policy](./infra/policies/round-robin-policy.xml) defines a round-robin load balancing strategy based on the following flow:

- On an inbound request, we check a cached `backend-counter` variable to determine which backend to use. If this doesn't exist, we create it and immediately set it to `0`.
- We then use the `backend-counter` variable to set the backend service to use for the request, as well as the required `api-key` header to use for the request to the Azure OpenAI Service.
- We then change the `backend-counter` to `1` and set it back to the cache.

- When processing the request at the backend, we provide a retry policy to handle any transient errors that may occur. This is to ensure that we don't lose any requests to the Azure OpenAI Service, and can swap to another backend if the current one is unavailable.
- The process follows the same logic as the inbound request, whereby we check the cached `backend-counter` variable to determine which backend to use and set the `api-key` header accordingly.

The [retry policy](https://learn.microsoft.com/en-us/azure/api-management/retry-policy) for the backend will only trigger if the status code of the response is `400` or greater, and will retry up to 3 times with a 5 second delay between each retry. If the first request immediately fails, the retry policy will ensure that the request is immediately refired to another backend.

In [ ]:
$deploymentOutputs = (az deployment sub create --name 'aoai-apim-loadbalancing' --location westeurope --template-file ./infra/main.bicep --parameters ./infra/main.bicepparam --query 'properties.outputs' -o json) | ConvertFrom-Json

### Get common outputs from Bicep deployment

The following will get the outputs from the Bicep deployment and set them as PowerShell variables for use in the Azure CLI commands.

> **Note:** All the outputs can be found at the bottom of the [main.bicep](./infra/main.bicep) file.

In [ ]:
$resourceGroup = $deploymentOutputs.resourceGroupInstance.value.name
$apiManagement = $deploymentOutputs.apiManagementInstance.value.name
$apiManagementGatewayUrl = $deploymentOutputs.apiManagementInstance.value.gatewayUrl
$apiManagementSubscriptionName = $deploymentOutputs.apiManagementInstance.value.subscriptionName

Write-Host "Resource group: $resourceGroup"
Write-Host "API Management: $apiManagement"
Write-Host "API Management Gateway URL: $apiManagementGatewayUrl"
Write-Host "API Management Subscription Name: $apiManagementSubscriptionName"

# Retrieve API Management subscription primary key
$apimSubscriptionEndpoint = "https://management.azure.com/subscriptions/$($subscriptionId)/resourceGroups/$($resourceGroup)/providers/Microsoft.ApiManagement/service/$($apiManagement)/subscriptions/$($apiManagementSubscriptionName)/listSecrets?api-version=2023-03-01-preview"
$apiManagementSubscription = (az rest --uri $apimSubscriptionEndpoint --method POST) | ConvertFrom-Json
$apiManagementSubscriptionKey = $apiManagementSubscription.primaryKey

## 3. Test API Management

Now that everything is configured, you can test the API Management API by making a request to the expected `/openai` endpoints.

This mechanism will work for directly communicating with Azure OpenAI Services using standard HTTP requests, as well as using libraries such as [Microsoft Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/overview/) and [LangChain](https://www.langchain.com).

The following examples showcase using standard HTTP requests to the API Management API.

### Creates a completion for the chat message

In [ ]:
$model = "gpt-35-turbo"
$apiVersion = "2023-07-01-preview"
$completionsEndpoint = $apiManagementGatewayUrl + "/openai/deployments/$($model)/chat/completions?api-version=$($apiVersion)"

# Define the request headers
$requestHeaders = @{
    "Ocp-Apim-Subscription-Key" = $apiManagementSubscriptionKey
}

# Define the request body
$requestBody = @{
    messages = @(
        @{
            role = "system"
            content = "You are a helpful AI assistant. You always try to provide accurate answers or follow up with another question if not."
        },
        @{
            role = "user"
            content = "What is the best way to get to London from Berlin?"
        }
    )
    max_tokens = 200
    temperature = 0.7
    top_p = 0.95
    frequency_penalty = 0
    presence_penalty = 0
}

$requestBodyString = ($requestBody | ConvertTo-Json -Depth 10 -Compress)

Write-Host "Posting request with URI $completionsEndpoint and body $requestBodyString"

$apiManagementResponse = Invoke-WebRequest -Uri $completionsEndpoint -Headers $requestHeaders -Method POST -Body $requestBodyString -ContentType "application/json"

Write-Host "Response: $apiManagementResponse"

# 4. Cleanup

The following will delete all the resources that were deployed as part of this notebook.

In [ ]:
az group delete --name $resourceGroup --yes --no-wait